In [131]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests

## Множественная проверка гипотез и ее коррекция
Допустим мы провели эксперимент с 6 вариантами и применили статистический критерий

Итого: 15 попарных проверок (6*6-6)/2

In [143]:
# Получили следующие значения p-value:
p_values = np.array([0.0029179437568, 0.009391279264, 0.011581441488, 
                     0.012616868376, 0.02839967164, 0.042167014336, 
                     0.04286582096, 0.0956598092, 0.10742021336, 
                     0.15927677808, 0.19332229592, 0.2456475724, 
                     0.254595706, 0.28608461424, 0.3626124616])

In [144]:
# Присвоим порядковые наименования для каждого из тестов
names = ["test" + str(i) for i in range(1, len(p_values) + 1)]
names

['test1',
 'test2',
 'test3',
 'test4',
 'test5',
 'test6',
 'test7',
 'test8',
 'test9',
 'test10',
 'test11',
 'test12',
 'test13',
 'test14',
 'test15']

In [145]:
df = pd.DataFrame({"names":names, "p_values":p_values})

In [146]:
df

,names,p_values
0,test1,0.002918
1,test2,0.009391
2,test3,0.011581
3,test4,0.012617
4,test5,0.028400
5,test6,0.042167
6,test7,0.042866
7,test8,0.095660
8,test9,0.107420
9,test10,0.159277


In [147]:
# Сортировка
df = df.sort_values(by="p_values")

In [148]:
df

,names,p_values
0,test1,0.002918
1,test2,0.009391
2,test3,0.011581
3,test4,0.012617
4,test5,0.028400
5,test6,0.042167
6,test7,0.042866
7,test8,0.095660
8,test9,0.107420
9,test10,0.159277


In [149]:
# коррекция Бонферонни: оптимизация FWER
_, bonferroni_p, _, _ = multipletests(df.p_values, method='bonferroni')
df["bonferroni_p"] = bonferroni_p
df

,names,p_values,bonferroni_p
0,test1,0.002918,0.043769
1,test2,0.009391,0.140869
2,test3,0.011581,0.173722
3,test4,0.012617,0.189253
4,test5,0.028400,0.425995
5,test6,0.042167,0.632505
6,test7,0.042866,0.642987
7,test8,0.095660,1.000000
8,test9,0.107420,1.000000
9,test10,0.159277,1.000000


In [150]:
# коррекция Холма: более мощная поправка, чем Бонферонни
_, holm_p, _, _ = multipletests(df.p_values, method='holm')
df["holm_p"] = holm_p
df

,names,p_values,bonferroni_p,holm_p
0,test1,0.002918,0.043769,0.043769
1,test2,0.009391,0.140869,0.131478
2,test3,0.011581,0.173722,0.150559
3,test4,0.012617,0.189253,0.151402
4,test5,0.028400,0.425995,0.312396
5,test6,0.042167,0.632505,0.421670
6,test7,0.042866,0.642987,0.421670
7,test8,0.095660,1.000000,0.765278
8,test9,0.107420,1.000000,0.765278
9,test10,0.159277,1.000000,0.955661


In [151]:
# коррекция Бенджамини-Хохберга: коррекция FDR, самая мощная из представленных поправок
_, bh_p, _, _ = multipletests(df.p_values, method='fdr_bh')
df["bh_p"] = bh_p
df

,names,p_values,bonferroni_p,holm_p,bh_p
0,test1,0.002918,0.043769,0.043769,0.043769
1,test2,0.009391,0.140869,0.131478,0.047313
2,test3,0.011581,0.173722,0.150559,0.047313
3,test4,0.012617,0.189253,0.151402,0.047313
4,test5,0.028400,0.425995,0.312396,0.085199
5,test6,0.042167,0.632505,0.421670,0.091855
6,test7,0.042866,0.642987,0.421670,0.091855
7,test8,0.095660,1.000000,0.765278,0.179034
8,test9,0.107420,1.000000,0.765278,0.179034
9,test10,0.159277,1.000000,0.955661,0.238915


In [ ]:
# визуализируем и сравним p_value между коррекциями
# Увидим, что только после BH поправка дает нам возможность отклонить нулевые гипотезы для 4 сравнений
# Поправки Бонферонни и Холма позволяют отклонять нулевую гипотезу только для 1 теста

In [ ]:
# res_p_values %>% 
#   gather(correction, p_value, -test) %>% # транспонируем табличку как нам требуется для визуализации
#   ggplot(aes(x = test, y = p_value)) +
#   geom_hline(yintercept = 0.05, color = "red") + # сделаем отсечку для уровня значимости 95%
#   geom_text(aes(label = round(p_value,3))) +
#   facet_grid(~correction) +
#   coord_flip()